In [1]:
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
from datetime import datetime

# --- Config ---
MONGO_URI = "mongodb://localhost:27017"  # ajusta se necessário
DB_NAME = "ctt2025"
COLL_NAME = "expresso_2025"

# Se True, apenas mostra o que faria (não escreve na BD)
DRY_RUN = False

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
collection = db[COLL_NAME]

def to_lowercase_keys(doc: dict):
    """
    Converte apenas as chaves de nível superior para minúsculas.
    Preserva os tipos dos valores.
    Mantém _id tal como está.
    Política de conflitos: a chave já em minúsculas prevalece.
    Retorna (novo_documento, conflitos_encontrados[lista de tuplos]).
    """
    new_doc = {"_id": doc["_id"]}
    conflicts = []

    for k, v in doc.items():
        if k == "_id":
            continue
        lk = k.lower()

        # Se já existe a chave em minúsculas, mantemos o que lá está
        if lk in new_doc and k != lk:
            # conflito: k -> lk, já existe lk
            conflicts.append((str(doc["_id"]), k, lk))
            continue

        new_doc[lk] = v

    return new_doc, conflicts


def normalize_collection(batch_size=1000, dry_run=False):
    total = collection.estimated_document_count()
    print(f"[{datetime.now().isoformat()}] Documentos estimados: {total}")

    # Iterador com batch_size; no_cursor_timeout para coleções grandes
    cursor = collection.find({}, no_cursor_timeout=True).batch_size(batch_size)

    processed = 0
    modified = 0
    conflicts_log = []

    try:
        for doc in cursor:
            new_doc, conflicts = to_lowercase_keys(doc)
            if conflicts:
                conflicts_log.extend(conflicts)

            # Se o documento já está igual, evita escrita desnecessária
            if all(k in doc and doc[k] == new_doc[k] for k in new_doc if k != "_id") and \
               all(k in new_doc for k in doc if k != "_id"):
                processed += 1
                if processed % 1000 == 0:
                    print(f"  - Processados: {processed} | Alterados: {modified}")
                continue

            if not dry_run:
                # replace_one preserva o _id e substitui o documento completo
                collection.replace_one({"_id": doc["_id"]}, new_doc, upsert=False)

            processed += 1
            modified += 1
            if processed % 1000 == 0:
                print(f"  - Processados: {processed} | Alterados: {modified}")

    finally:
        cursor.close()

    print(f"[{datetime.now().isoformat()}] Terminado. Processados: {processed} | Alterados: {modified}")

    if conflicts_log:
        print(f"Conflitos (mostrando até 20):")
        for item in conflicts_log[:20]:
            _id, original_key, lowered_key = item
            print(f"  _id={_id} : '{original_key}' -> '{lowered_key}' (ignorado porque '{lowered_key}' já existia)")
        if len(conflicts_log) > 20:
            print(f"  ... e mais {len(conflicts_log) - 20} conflitos similares.")
    else:
        print("Sem conflitos de chaves.")

if __name__ == "__main__":
    print(f"Normalizar chaves para minúsculas em {DB_NAME}.{COLL_NAME} | DRY_RUN={DRY_RUN}")
    normalize_collection(dry_run=DRY_RUN)


Normalizar chaves para minúsculas em ctt2025.expresso_2025 | DRY_RUN=False
[2025-08-26T11:16:44.743581] Documentos estimados: 23345487
  - Processados: 1000 | Alterados: 0
  - Processados: 2000 | Alterados: 0
  - Processados: 3000 | Alterados: 0
  - Processados: 4000 | Alterados: 0
  - Processados: 5000 | Alterados: 0
  - Processados: 6000 | Alterados: 0
  - Processados: 7000 | Alterados: 0
  - Processados: 8000 | Alterados: 0


C:\Users\1420844\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pymongo\synchronous\collection.py:1945: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


  - Processados: 9000 | Alterados: 0
  - Processados: 10000 | Alterados: 0
  - Processados: 11000 | Alterados: 0
  - Processados: 12000 | Alterados: 0
  - Processados: 13000 | Alterados: 0
  - Processados: 14000 | Alterados: 0
  - Processados: 15000 | Alterados: 0
  - Processados: 16000 | Alterados: 0
  - Processados: 17000 | Alterados: 0
  - Processados: 18000 | Alterados: 0
  - Processados: 19000 | Alterados: 0
  - Processados: 20000 | Alterados: 0
  - Processados: 21000 | Alterados: 0
  - Processados: 22000 | Alterados: 0
  - Processados: 23000 | Alterados: 0
  - Processados: 24000 | Alterados: 0
  - Processados: 25000 | Alterados: 0
  - Processados: 26000 | Alterados: 0
  - Processados: 27000 | Alterados: 0
  - Processados: 28000 | Alterados: 0
  - Processados: 29000 | Alterados: 0
  - Processados: 30000 | Alterados: 0
  - Processados: 31000 | Alterados: 0
  - Processados: 32000 | Alterados: 0
  - Processados: 33000 | Alterados: 0
  - Processados: 34000 | Alterados: 0
  - Processad

In [2]:
from pymongo import MongoClient
from datetime import datetime

# --- Config ---
MONGO_URI = "mongodb://localhost:27017"  # ajusta se necessário
DB_NAME = "ctt2025"
COLL_NAME = "expresso_2025"

# Se True, apenas mostra o que faria (não escreve na BD)
DRY_RUN = False

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
collection = db[COLL_NAME]

def force_values_to_string_except_date(doc: dict):
    """
    Converte todos os valores do nível superior em string,
    exceto _id (mantém ObjectId) e data_criacao (mantém Date).
    """
    new_doc = {"_id": doc["_id"]}
    for k, v in doc.items():
        if k == "_id":
            continue
        if k == "data_criacao":
            # força para datetime se vier como string
            if isinstance(v, str):
                try:
                    new_doc[k] = datetime.fromisoformat(v.replace("Z", "+00:00"))
                except Exception:
                    new_doc[k] = v  # se não conseguir converter, deixa como está
            else:
                new_doc[k] = v
        else:
            if v is None:
                new_doc[k] = ""  # substitui null por string vazia
            else:
                new_doc[k] = str(v)
    return new_doc

def normalize_collection(batch_size=1000, dry_run=False):
    total = collection.estimated_document_count()
    print(f"[{datetime.now().isoformat()}] Documentos estimados: {total}")

    cursor = collection.find({}, no_cursor_timeout=True).batch_size(batch_size)

    processed = 0
    modified = 0

    try:
        for doc in cursor:
            new_doc = force_values_to_string_except_date(doc)

            # Evita update se o documento já estiver igual
            same = True
            for k, v in new_doc.items():
                if k == "_id":
                    continue
                if doc.get(k) != v:
                    same = False
                    break

            if same:
                processed += 1
                continue

            if not dry_run:
                collection.replace_one({"_id": doc["_id"]}, new_doc, upsert=False)

            processed += 1
            modified += 1

            if processed % 1000 == 0:
                print(f"  - Processados: {processed} | Alterados: {modified}")

    finally:
        cursor.close()

    print(f"[{datetime.now().isoformat()}] Terminado. Processados: {processed} | Alterados: {modified}")

if __name__ == "__main__":
    print(f"Forçar valores para string (exceto data_criacao) em {DB_NAME}.{COLL_NAME} | DRY_RUN={DRY_RUN}")
    normalize_collection(dry_run=DRY_RUN)


Forçar valores para string (exceto data_criacao) em ctt2025.expresso_2025 | DRY_RUN=False
[2025-08-26T14:11:44.056712] Documentos estimados: 23345487
  - Processados: 28000 | Alterados: 1484
  - Processados: 31000 | Alterados: 1830
  - Processados: 33000 | Alterados: 2008
  - Processados: 47000 | Alterados: 3224
  - Processados: 57000 | Alterados: 4258
  - Processados: 64000 | Alterados: 4844
  - Processados: 68000 | Alterados: 5329
  - Processados: 69000 | Alterados: 5492
  - Processados: 70000 | Alterados: 5634
  - Processados: 103000 | Alterados: 8712
  - Processados: 123000 | Alterados: 10159
  - Processados: 135000 | Alterados: 11195
  - Processados: 143000 | Alterados: 12083
  - Processados: 153000 | Alterados: 13625
  - Processados: 156000 | Alterados: 13807
  - Processados: 161000 | Alterados: 14187
  - Processados: 165000 | Alterados: 14567
  - Processados: 174000 | Alterados: 15579
  - Processados: 180000 | Alterados: 16165
  - Processados: 184000 | Alterados: 16463
  - Proce